In [1]:
import numpy as np

# Load the dataset
data = np.load(r'C:\Project\Bird_drone_CWT\RPC12_Input_For_Participants\datasets\data_SAAB_SIRS_77GHz_FMCW.npy', allow_pickle=True)

# Extract relevant data: complex-valued data matrix from Column 2
complex_data = data[:, 1] 

In [2]:
first_row = data[0]
complex_valued_data = first_row[1]
complex_valued_data.shape

(1280, 1228)

In [3]:
import numpy as np

# Ensure complex_valued_data is a numpy array
complex_valued_data = np.array(complex_valued_data, dtype=np.complex128)

# Separate real and imaginary parts
real_part = np.real(complex_valued_data)
imaginary_part = np.imag(complex_valued_data)

# Combine real and imaginary parts into a 2D array
combined_data = np.column_stack((real_part, imaginary_part))

# Save to a text file
np.savetxt('complex_data.txt', combined_data, fmt='%.6f', header='Real Part, Imaginary Part')


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
from scipy.signal import find_peaks
import cv2
import os
from scipy.signal import resample

# Define directories for saving and loading images
save_dir = 'scalogram_images'
os.makedirs(save_dir, exist_ok=True)

# Load the dataset


# Extract the complex-valued radar signal data (Column 2)
complex_data = data[:, 1]

# Define CWT parameters
wavelet = 'cmor'  # Complex Morlet wavelet
scales = np.arange(1, 128)  # Reduce scale range to save memory
fs = 17000  # Original sampling frequency (radar PRF)
downsample_factor = 4  # Factor to downsample by

# Function to perform CWT and save scalogram images
def save_scalogram_image(segment, index):
    # Downsample the signal to reduce memory usage
    segment_downsampled = resample(segment.flatten(), len(segment) // downsample_factor)
    
    # Perform CWT on downsampled signal
    coef, _ = pywt.cwt(segment_downsampled, scales, wavelet)
    
    # Absolute values for magnitude representation
    scalogram = np.abs(coef)
    
    # Save scalogram as image
    plt.imshow(scalogram, aspect='auto', cmap='jet', origin='lower')
    plt.colorbar()
    plt.title(f'Scalogram - Segment {index + 1}')
    plt.savefig(os.path.join(save_dir, f'scalogram_{index + 1}.png'))
    plt.close()

# Save all scalograms as images
for i, segment in enumerate(complex_data):
    print(f'Saving scalogram image for segment {i + 1}/{len(complex_data)}...')
    save_scalogram_image(segment, i)

# Function to extract features from scalogram images
def extract_features_from_image(image_path):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # If image loading fails, return default features
    if img is None:
        return {
            'frequency_peaks': 0,
            'avg_peak_height': 0,
            'periodicity': 0,
            'modulation_variance': 0,
        }
    
    # Convert image to float and normalize
    img = img.astype(float)
    img /= np.max(img)
    
    # Compute features from the image
    avg_frequency_magnitude = np.mean(img, axis=1)
    peaks, _ = find_peaks(avg_frequency_magnitude, height=np.max(avg_frequency_magnitude) * 0.5)
    
    peak_intervals = np.diff(peaks)  # Differences between peak indices
    periodicity = np.std(peak_intervals) if len(peak_intervals) > 1 else 0

    modulation_pattern = np.var(img, axis=1)

    features = {
        'frequency_peaks': len(peaks),
        'avg_peak_height': np.mean(avg_frequency_magnitude[peaks]) if peaks.size > 0 else 0,
        'periodicity': periodicity,
        'modulation_variance': np.mean(modulation_pattern),
    }

    return features

# Extract features from all scalogram images
all_features = []
for i in range(len(complex_data)):
    image_path = os.path.join(save_dir, f'scalogram_{i + 1}.png')
    print(f'Extracting features from image {i + 1}/{len(complex_data)}...')
    features = extract_features_from_image(image_path)
    all_features.append(features)

# Example: Print features of the first segment
print("Features from the first image:", all_features[0])


Saving scalogram image for segment 1/130...


c:\Users\dhars\AppData\Local\Programs\Python\Python312\Lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)


Saving scalogram image for segment 2/130...
Saving scalogram image for segment 3/130...
Saving scalogram image for segment 4/130...
Saving scalogram image for segment 5/130...
Saving scalogram image for segment 6/130...
Saving scalogram image for segment 7/130...
Saving scalogram image for segment 8/130...
Saving scalogram image for segment 9/130...
Saving scalogram image for segment 10/130...
Saving scalogram image for segment 11/130...
Saving scalogram image for segment 12/130...
Saving scalogram image for segment 13/130...
Saving scalogram image for segment 14/130...
Saving scalogram image for segment 15/130...
Saving scalogram image for segment 16/130...
Saving scalogram image for segment 17/130...
Saving scalogram image for segment 18/130...
Saving scalogram image for segment 19/130...
Saving scalogram image for segment 20/130...
Saving scalogram image for segment 21/130...
Saving scalogram image for segment 22/130...
Saving scalogram image for segment 23/130...
Saving scalogram i

In [5]:
import numpy as np
import pandas as pd

# Extract labels from Column 1
labels = data[:, 0]

# Ensure labels are plain strings by accessing the first element in each array
labels = [str(label[0]) if isinstance(label, (list, np.ndarray)) else str(label) for label in labels]

# Print unique labels in the dataset
print("Unique labels in the dataset:", np.unique(labels))

# Create a new mapping for simplified classification
simplified_label_mapping = {
    'D1': 'drone', 'D2': 'drone', 'D3': 'drone', 'D4': 'drone', 
    'D5': 'drone', 'D6': 'drone',  # All drone types
    'human_walk': 'human', 'human_run': 'human',  # Human activities
    'seagull': 'bird', 'pigeon': 'bird', 'raven': 'bird', 'black-headed gull': 'bird',
    'seagull and black-headed gull': 'bird', 'heron': 'bird',  # All bird types
    'CR': None  # Ignore corner reflector samples or map them to a category if needed
}

# Apply the simplified label mapping
simplified_labels = np.array([simplified_label_mapping.get(label, None) for label in labels])

# Print unique labels after mapping
print("Unique labels after mapping:", np.unique([label for label in simplified_labels if label is not None]))

# Remove rows with 'None' labels (corner reflector or any unwanted data)
valid_indices = [i for i, label in enumerate(simplified_labels) if label is not None]
simplified_features = [all_features[i] for i in valid_indices]

# Check if we have valid data after filtering
print(f"Number of valid samples: {len(valid_indices)}")

if len(valid_indices) == 0:
    print("No valid data found after filtering. Check the label mapping or data extraction.")
else:
    # Convert the list of features to a pandas DataFrame
    features_df = pd.DataFrame(simplified_features)

    # Add the simplified labels as a new column
    features_df['label'] = simplified_labels[valid_indices]

    # Display the first few rows to check the combined DataFrame
    print(features_df.head())

    # Convert the DataFrame to a NumPy array (if needed for certain ML libraries)
    features_array = features_df.to_numpy()

    # Separate features (X) and labels (y)
    X = features_array[:, :-1]  # All columns except the last
    y = features_array[:, -1]   # The last column (simplified label)

    # Display the shapes of X and y to confirm
    print("Feature matrix shape:", X.shape)
    print("Labels shape:", y.shape)

   

Unique labels in the dataset: ['CR' 'D1' 'D2' 'D3' 'D4' 'D5' 'D6' 'black-headed gull' 'heron'
 'human_run' 'human_walk' 'pigeon' 'raven' 'seagull'
 'seagull and black-headed gull']
Unique labels after mapping: ['bird' 'drone' 'human']
Number of valid samples: 111
   frequency_peaks  avg_peak_height  periodicity  modulation_variance  label
0               39         0.749798    16.952108             0.069709  drone
1               43         0.784036    12.635300             0.062910  drone
2               50         0.729242     9.593791             0.075981  drone
3               60         0.806211     9.556407             0.055883  drone
4               54         0.725570    11.925782             0.089771  drone
Feature matrix shape: (111, 4)
Labels shape: (111,)


In [6]:
from scipy.stats import entropy, skew, kurtosis
import numpy as np

def extract_features(signal):
    # Calculate entropy, skewness, and kurtosis
    signal_entropy = entropy(np.abs(signal) + 1e-10)  # Adding small value to avoid log(0) issue in entropy
    signal_skewness = skew(np.abs(signal))
    signal_kurtosis = kurtosis(np.abs(signal))

    # Calculate band power (example for a specific frequency band)
    band_power = np.sum(np.abs(signal)**2)

    # Ensure all extracted features are scalar values
    return [signal_entropy, signal_skewness, signal_kurtosis, band_power]

# Apply feature extraction to all segments
features = [extract_features(segment.flatten()) for segment in complex_data]

# Convert to numpy array after ensuring all elements are consistent in length
X_new_features = np.array(features)


In [7]:
# Check the shapes of both arrays
print(f"Shape of X_old: {X.shape}")
print(f"Shape of X_new_features: {X_new_features.shape}")

# Trim or align data
if X.shape[0] < X_new_features.shape[0]:
    X_new_features = X_new_features[:X.shape[0], :]
elif X.shape[0] > X_new_features.shape[0]:
    X_old = X[:X_new_features.shape[0], :]

# Horizontally stack the features after ensuring they have the same number of rows
X_combined = np.hstack((X, X_new_features))


Shape of X_old: (111, 4)
Shape of X_new_features: (130, 4)


In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

# Example: Assuming 'X' is your feature matrix
scaler = StandardScaler()  # Use MinMaxScaler() for normalization
X_scaled = scaler.fit_transform(X_combined)  # Normalizing/Standardizing features


In [9]:
from imblearn.over_sampling import SMOTE

# Assuming X_scaled is your feature matrix and y is your target vector
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)


In [10]:
y_resampled[0]

'drone'

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense

# Autoencoder Definition
def build_autoencoder(input_dim):
    # Define Encoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(128, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    
    # Define Decoder
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # Build Autoencoder
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder, encoder

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build and train autoencoder
input_dim = X_train_scaled.shape[1]
autoencoder, encoder = build_autoencoder(input_dim)

# Train the autoencoder
autoencoder.fit(X_train_scaled, X_train_scaled, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

# Encode the data
X_train_encoded = encoder.predict(X_train_scaled)
X_test_encoded = encoder.predict(X_test_scaled)

# Function for Bayesian Optimization of SVM
def bayesian_optimization_svm(X, y):
    # SVM Model
    svm = SVC()

    # Define parameter search space
    param_space = {
        'C': (1e-6, 1e+2, 'log-uniform'),  # Regularization parameter
        'gamma': (1e-6, 1e+1, 'log-uniform'),  # Kernel coefficient
        'kernel': ['rbf']  # Using RBF kernel
    }

    # Bayesian Search with cross-validation
    search = BayesSearchCV(svm, param_space, n_iter=32, cv=5, random_state=42, n_jobs=-1)
    search.fit(X, y)

    return search

# Perform Bayesian Optimization for SVM
search = bayesian_optimization_svm(X_train_encoded, y_train)

# Evaluate on the test set
y_pred = search.predict(X_test_encoded)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 1.1699 - val_loss: 1.2433
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2007 - val_loss: 1.2087
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.1851 - val_loss: 1.1564
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.1430 - val_loss: 1.0841
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.0416 - val_loss: 1.0031
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.9533 - val_loss: 0.9321
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.8583 - val_loss: 0.8763
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.7952 - val_loss: 0.8283
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.7962 - val_loss: 0.7904
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.7442 - val_loss: 0.7692
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6914 - val_loss: 0.7566
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6409 - val_loss: 0.7484
E

In [12]:
# Save the autoencoder model
autoencoder.save('autoencoder_model.h5')

# Save the encoder part separately if needed
encoder.save('encoder_model.h5')


In [13]:
import joblib

# Save the SVM model after Bayesian Optimization
joblib.dump(search, 'svm_model.pkl')


['svm_model.pkl']

In [15]:
print(features_df.tail(100))

     frequency_peaks  avg_peak_height  periodicity  modulation_variance  label
11                55         0.741952     8.974740             0.071427  drone
12                40         0.759279    20.557260             0.083368  drone
13                44         0.732383    12.985327             0.079751  drone
14                55         0.733543    11.245972             0.082059  drone
15                54         0.797218    15.445514             0.072113  drone
..               ...              ...          ...                  ...    ...
106               19         0.700121    49.645658             0.126285   bird
107               55         0.610741     5.603374             0.108210   bird
108               17         0.725300    32.618773             0.125654   bird
109               76         0.651357     3.362909             0.093119   bird
110               16         0.757716    66.485554             0.132315   bird

[100 rows x 5 columns]


In [16]:
unique_labels, counts = np.unique(y, return_counts=True)

# Print results
for label, count in zip(unique_labels, counts):
    print(f"Label: {label}, Count: {count}")

Label: bird, Count: 56
Label: drone, Count: 44
Label: human, Count: 11


In [17]:
first_row = X_combined[0]

first_row


array([39, 0.7497979512317747, 16.952107869490348, 0.06970923415639042,
       13.47753931266451, 4.501689334353405, 30.485283292779023,
       7659506.093375225], dtype=object)

In [14]:
import numpy as np
import pywt
import cv2
from scipy.signal import resample

# Load your new radar complex-valued data
# Assuming `new_complex_data` is a numpy array containing complex radar signals
new_complex_data = complex_data[0]  # Load your new radar complex-valued signal here

# Define CWT parameters
wavelet = 'cmor'
scales = np.arange(1, 128)
downsample_factor = 4

# Function to perform CWT and save scalogram images (can skip saving if not needed)
def preprocess_radar_data(segment):
    # Downsample the signal
    segment_downsampled = resample(segment.flatten(), len(segment) // downsample_factor)
    
    # Perform CWT
    coef, _ = pywt.cwt(segment_downsampled, scales, wavelet)
    
    # Absolute value for magnitude representation (scalogram)
    scalogram = np.abs(coef)
    
    return scalogram

# Extract scalogram features from new radar data
def extract_features_from_scalogram(scalogram):
    # Example feature extraction from scalogram (you can customize this)
    avg_frequency_magnitude = np.mean(scalogram, axis=1)
    peaks, _ = find_peaks(avg_frequency_magnitude, height=np.max(avg_frequency_magnitude) * 0.5)
    
    peak_intervals = np.diff(peaks)
    periodicity = np.std(peak_intervals) if len(peak_intervals) > 1 else 0
    modulation_pattern = np.var(scalogram, axis=1)
    
    features = [
        len(peaks),
        np.mean(avg_frequency_magnitude[peaks]) if peaks.size > 0 else 0,
        periodicity,
        np.mean(modulation_pattern),
    ]
    
    return features

# Preprocess each new radar segment
new_features = []
for segment in new_complex_data:
    scalogram = preprocess_radar_data(segment)
    features = extract_features_from_scalogram(scalogram)
    new_features.append(features)

# Convert to numpy array
new_features = np.array(new_features)


c:\Users\dhars\AppData\Local\Programs\Python\Python312\Lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)


In [16]:
from sklearn.preprocessing import StandardScaler
import joblib

# Assuming X_train is your training feature matrix
scaler = StandardScaler()

# Fit the scaler to your training data
scaler.fit(X_train)

# Save the scaler to a file
joblib.dump(scaler, 'scaler.pkl')

print("Scaler saved to 'scaler.pkl'")


Scaler saved to 'scaler.pkl'
